### Decision tree

In [45]:
# import dependencies
import numpy as np
from collections import Counter
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [66]:
# entropy function as a global function
def entropy(y):
    # calculate the number of occurences of all class labels
    hist = np.bincount(y)
    ps = hist/ len(y)
    # apply the formula for entropy
    return -np.sum([p*np.log2(p) for p in ps if p>0])

# a helper class to store information of the node
class Node:
    def __init__(self, feature=None, threshold=None,left=None,right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
    
    # a helper function to determine if we are at the leafnode
    def is_leaf_node(self):
        return self.value is not None

# decision tree class
class ScratchDecesionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=100, n_feats=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        # a root so that we can know where we started
        self.root = None
        
    def fit(self, X,y):
        # grow tree
        # apply a safety check
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self._grow_tree(X, y)
        
    # function to grow the tree
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        
        # apply the stopping criteria of our growing tree
        if(depth >= self.max_depth
          or n_labels ==1
          or n_samples < self.min_samples_split):
            # if the condition is true that would mean that we are at the leaf node
            # check for the common label in our node
            leaf_value = self._most_common_label(y)
            # use our class to store the node information
            return Node(value=leaf_value)
        
        # if we did not meet the stopping criteria
        feat_idxs = np.random.choice(n_features, self.n_feats,replace=False)
        
        # do a greedy search
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
        left_idxs, right_idxs = self._split(X[:,best_feat], best_thresh)
        
        # continue growing
        left = self._grow_tree(X[left_idxs,:], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs,:], y[right_idxs], depth+1)
        
        node = Node(best_feat, best_thresh, left, right)
        
        self.impurity = self._gini(node)
        return node
        
        
        
    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -1
        split_idxs, split_thresh = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            # lets go over all threshold
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                # calculating the information gain
                gain = self._information_gain(y, X_column, threshold)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold
        return split_idx, split_thresh
    
    # a function to calculate the information gain
    def _information_gain(self, y, X_column, split_thresh):
        # calculate the parent entropy
        parent_entropy = entropy(y)
        
        # generate split
        left_idxs, right_idxs = self._split(X_column, split_thresh)
        
        if len(left_idxs) == 0 or len(right_idxs)==0:
            return 0
        
        # calculate the weighted average child entropy
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) *e_r
        
        # apply the formula for information gain
        ig = parent_entropy - child_entropy
        return ig
        
    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs
    
    def predict(self, X):
        # traverse our tree
        return np.array([self._traverse_tree(x, self.root) for x in X])
    
    # a function helper for prediction
    def _traverse_tree(self, x, node):
        # check for the stopping criteria
        if node.is_leaf_node():
            return node.value
        # check if we go from left or right
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
    
    # function to check the common label in our node
    def _most_common_label(self, y):
        # calculating the number of occurencies in y using counter
        counter = Counter(y)
        # getting the value of the most common label
        most_common = counter.most_common(1)[0][0]
        return most_common
    
    # gini impurity
    def _gini(self,node):
        unique,count = np.unique(node,return_counts=True)
        tmp = 0
        for i in unique:
            tmp += (count[unique==i]/np.sum(count))**2

        return 1-tmp

In [8]:
data = datasets.load_breast_cancer()
X = data.data
y = data.target

In [9]:
# splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [67]:
# training our classifier
clf = ScratchDecesionTreeClassifier(max_depth=2)
clf.fit(X_train, y_train)

In [68]:
# predicting
y_pred = clf.predict(X_test)

In [69]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('accuracy',accuracy)
print('precision',precision)
print('recall',recall)
print('f1',f1)

accuracy 0.9035087719298246
precision 0.953125
recall 0.8840579710144928
f1 0.9172932330827068


### Problem 1

In [54]:
clf.impurity

array([0.])

### Problem 2

Information gain function, implemented in the classifier above

### Problem 3